# Artistic Style Transfer 

This Colab contains a replication of the artistic style transfer proposed by Leon A. Gatys, Alexander S. Ecker and Matthias Bethge in their paper 'A Neural Algorithm of Artistic Style' (2015).

*Replication by Marisa Wodrich and Johanna Linkemeyer*

##**1 Introduction**

Hier kommt ein bisschen  Text, was machen wir so und wie geht das und wieso ist das hart cool?

##**2 Data**

In this section, the data for our style transfer is loaded. Also, the functions to preprocess our data for the network are defined here. The data we use are images. More precisely, for our artistic style transfer, we require content and style images. Content images can be basically any image. We chose the Neckarfront image from the paper we replicate and, additionally, some other images we found on the internet to test the effectiveness of the style transfer on other images, too:
* Image of the Neckarfront in Tübingen (Original paper by Gatys and colleagues)
* A Field of Sunflowers
* A Puppy on a Field
* A Lake in Front of a Mountain

For the style images, we chose the five style images from the original paper and added two more that we liked particularly. The style images provided from us are:
* The Starry Night by Vincent van Gogh (1889) *- Post-Impressionism*
* The Scream by Edvard Munch (1893) *- Expressionism*
* Cafeterrasse am Abend by Vincent van Gogh (1888) *- Post-Impressionism*
* Style from the Artist James Rizzy *-* 
* The Shipwreck of the Minotaur by William Turner (1810) *- Romantic*
* Composition VII by Wassily Kadinsky (1913) *- Abstract Art*
* Femme Nue Assise by Pablo Picasso (1910) *- Cubism*


###**2.1 Setup**

This section is for setup purposes. All necessary packages are imported and the required directories to store the data are created.

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, Model

from skimage import io
import numpy as np
import os 
import matplotlib.pyplot as plt
import time
import functools
from PIL import Image
import cv2
from IPython.display import clear_output

import skimage

In [ ]:
def make_dirs(paths):
  """
  Creates directories from given paths if they do not exist.

  paths: (list) list of paths to directories
  return: None
  """
  
  for path in paths:

    if not os.path.exists(path):
      os.makedirs(path)

In [ ]:
# Define paths directories for images (subdivided into style, content and 
# generated images) if they do not exist.
path = 'images'
path_style = os.path.join(path, 'style')
path_content = os.path.join(path, 'content')
path_generated = os.path.join(path, 'generated')

# Create directories from predefined paths.
make_dirs([path, path_style, path_content, path_generated])

###**2.2 Load and Visualize Images**

The following sections contains all functions to load images and visualize them.

There is the option to add an image from your own link or your computer if you have a preferred image for style transfer. You can find the instructions to use your own image in the section **2.1.2 Load Images**.



####**2.2.1 Helper Functions**

In [ ]:
# This dictionary stores the image names and their respective description.
TITLE_DICT = {}

In [ ]:
def update_dictionary(image_name, image_description):
  """
  Updates the dictionary that stores the image descriptions to use them 
  as titles for visualization.
  
  image_name: (string) name of image file
  image_description: (string) description of image, e.g., its full title
  return: None
  """

  TITLE_DICT[image_name] = image_description

In [ ]:
def test_jpg_or_png(string_to_test):
  """
  Tests whether a given string ends with '.jpg' or '.png'.

  string_to_test: (string) string of which the ending should be tested
  return: (bool) True if string ends with '.jpg' or '.png', False else
  """

  return string_to_test.endswith('.jpg') or string_to_test.endswith('.png')

In [ ]:
def load_image_from_link(link, image_name, image_description, style_image=True):
  """
  Load an image from a given string and save it with a given file name in 
  the style or content image directory. Also, the TITLE_DICT is updated.
  
  link: (string) link to the image to load
  image_name: (string) name of the image file
  image_description: (string) description of the image
  style_image: (bool) whether to load a style or content image - True for 
      style image, False for content image
  return: None
  """

  # Test if given image_name is valid
  assert test_jpg_or_png(image_name), 'Please provide the image name with the \
      ending \'.jpg\' or \'.png\'.'
  assert test_jpg_or_png(link), 'Please provide a valid link. The given link \
      is not a link to an image.'

  # If style_image is true, the image should be saved in the directory 
  # for style images, else saved in the directory for content images.
  if style_image:
    image_type = 'style'
  else:
    image_type = 'content'

  path2dir = os.path.join('/content', 'images', image_type)

  image_path = os.path.join(path2dir, image_name)

  # Load image from link and convert to RGB color space
  image = io.imread(link)
  image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  # Save loaded image in given path
  cv2.imwrite(image_path, image_rgb)

  # Update the dictionary with image description (for plotting all 
  # loaded images)
  update_dictionary(image_name, image_description)

####**2.2.2 Load Images**

Here, the functions above are used to load some images that can/ will later be used for style transfer. We provide seven style images and four content images. The images from the paper by Gatys and colleagues are included to allow a comparison of our results to theirs.

If you would like to use a style or content image of your choice from a link, feel free to use the 

>**load_image_from_link(link, image_name, image_description, style_image)** 

function. For the parameters, please adapt them to your needs in the following way:

* **link**: Here, you should insert the link to your image. Please use the link that explicitly leads to the image, not the website you obtain it from. To check, this link should end with **.jpg** or **.png**. For some images, even though you provide the correct link, this method of loading the image won't work. You can still use the image by saving it on your device and uploading it manually to the respective folder.

* **image_name**: Here, you should insert the name you want to give the image. Please check, that the name ends with **.jpg** or **.png**.

* **image_description**: Please provide a short description of the image. This description will be used as a title in the image visualization.

* **style_image**: Depending on whether you would like to add a new style or content image, set this boolean to *true* or *false*. Choose *true* if you add a style image, and *false* for a content image.



In [ ]:
# ---- STYLE IMAGES ----

# The Starry Night by Vincent van Gogh (1889)
load_image_from_link('https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1280px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg',
                     'starry_night.jpg',
                     'Starry Night',
                     style_image=True)

# The Scream by Edvard Munch (1893)
load_image_from_link('https://upload.wikimedia.org/wikipedia/commons/thumb/c/c5/Edvard_Munch%2C_1893%2C_The_Scream%2C_oil%2C_tempera_and_pastel_on_cardboard%2C_91_x_73_cm%2C_National_Gallery_of_Norway.jpg/300px-Edvard_Munch%2C_1893%2C_The_Scream%2C_oil%2C_tempera_and_pastel_on_cardboard%2C_91_x_73_cm%2C_National_Gallery_of_Norway.jpg',
                     'the_scream.jpg',
                     'The Scream',
                     style_image=True)

# Cafeterrasse am Abend by Vincent van Gogh (1888)
load_image_from_link('https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Gogh4.jpg/300px-Gogh4.jpg',
                     'cafeterrasse_am_abend.jpg',
                     'Caféterrasse am Abend',
                     style_image=True)

# Style from the Artist James Rizzy
load_image_from_link('http://www.james-rizzi.com/wp-content/uploads/pictures/cache/2007_03_000_KeepingBusyInARizziCity_800_600.jpg',
                     'rizzy.jpg',
                     'Rizzy Style',
                     style_image=True)

# The Shipwreck of the Minotaur by William Turner (1810)
load_image_from_link('https://upload.wikimedia.org/wikipedia/commons/2/2e/Shipwreck_turner.jpg',
                     'shipwreck.jpg',
                     'The Shipwreck of the Minotaur',
                     style_image=True)

# Composition VII by Wassily Kadinsky (1913)
load_image_from_link('https://upload.wikimedia.org/wikipedia/commons/b/b4/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg',
                     'composition_vii.jpg',
                     'Composition VII',
                     style_image=True)

# Femme Nue Assise by Pablo Picasso (1910)
load_image_from_link('https://az334033.vo.msecnd.net/images-7/seated-nude-femme-nue-assise-pablo-picasso-1909-f9095482.jpg',
                     'femme_nue_assise.jpg',
                     'Femme nue assise',
                     style_image=True)

# ---- CONTENT IMAGES ----

# Image of a Field of Sunflowers
load_image_from_link('https://www.myhomebook.de/data/uploads/2020/02/gettyimages-1141659565-1024x683.jpg',
                     'sunflower.jpg',
                     'Sunflower',
                     style_image=False)

# Image of a Lake in Front of a Mountain
load_image_from_link('https://www.scinexx.de/wp-content/uploads/0/1/01-35131-nukliduhr01.jpg',
                     'mountain.jpg',
                     'Mountain',
                     style_image=False)

# Image of a Puppy on a Field
load_image_from_link('https://www.mera-petfood.com/files/_processed_/a/4/csm_iStock-521697453_7570f7a9b6.jpg',
                     'puppy.jpg',
                     'Puppy',
                     style_image=False)

# Image of the Neckarfront in Tübingen
load_image_from_link('https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg',
                     'neckarfront.jpg',
                     'Neckarfront',
                     style_image=False)


####**2.2.3 Image Visualization**

Here, one can take a look at the loaded data. Running the following two code blocks will visualize all style and content images that are in the respective folder.

In [ ]:
def visualize_images(path):
  """
  This function creates a plot of all images contained in the directory (given 
  path).
  
  path: (string) path to directory of all images to be plotted
  return: None
  """
  
  assert os.path.exists(path), 'Given path does not exist.'
  assert os.listdir(path), 'Given path is empty.'

  path_content = os.listdir(path)

  # Remove files that are not images (This can be seen as a step of caution 
  # but usually path_content and imgs should not differ because we only load 
  # images to the folder).
  imgs = [element for element in path_content if element.endswith('.jpg') or \
          element.endswith('.png')]

  # Test if images were found, so plotting does not run into errors.
  assert imgs, 'No images found in given path. Please repeat the loading process.'

  if len(imgs) > 1:
    _, ax = plt.subplots(1, len(imgs), figsize=(len(imgs)*3,len(imgs)*2))
  elif len(imgs) == 1:
    _, ax = plt.subplots(1, figsize=(len(imgs)*3,len(imgs)*2))

  # Iterate over list of image references, load images 
  # and plot them beside each other.
  for i, img in enumerate(imgs):

    # If image name is listed in title dictionary, search for corresponding 
    # image title. Else, just use the filename without the ending as title.
    if img in TITLE_DICT:
      img_name = TITLE_DICT[img]
    else:
      img_name = img.split('.')[0]

    # Load image
    img = Image.open(os.path.join(path, img))

    # Plot images (Special case for plotting if there is only one image in the 
    # given path.)
    if len(imgs) > 1:
      ax[i].imshow(img)
      ax[i].axis('off')
      ax[i].title.set_text(img_name)
    elif len(imgs) == 1:
      ax.imshow(img)
      ax.axis('off')
      ax.title.set_text(img_name)

  plt.tight_layout()
  plt.show()

In [ ]:
print('STYLE IMAGES')
visualize_images(path_style)

print('\nCONTENT IMAGES')
visualize_images(path_content)

###**2.3 Preprocessing Functions**

For this project, we will use pre-trained layers from VGG19, a Convolutional Neural Network (CNN) for object detection. Input images to VGG19 must be preprocessed in the following way:

* Size of images: 224 x 224 x 3
* Color range: [0, 255]
* Value type: float 32
* Colorspace: BGR
* Normalization (zero-centering) per channel (B: 103.939, G: 116.779, R: 123.68)

*This information is obtained from [here](https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg19/preprocess_input) and [here](https://towardsdatascience.com/extract-features-visualize-filters-and-feature-maps-in-vgg16-and-vgg19-cnn-models-d2da6333edd0) and not provided in the paper by Gatys and colleagues.*

NOTE: We do not process all data right away because we can do it easily when running the style transfer later.

In [ ]:
def img2tensor(path):
  """
  Loads an image from a given path and preprocesses it for usage for the 
  VGG19 network as described above.

  path: (string) path to an image
  return: (tensor) preprocessed image as tensor
  """

  # Load image as BGR, float32 image in the range [0,255]
  img = cv2.imread(path).astype(np.float32)

  # Save information about the original shape
  orig_shape = img.shape 

  # Normalize (zero-center) the values according to imagenet mean of 
  # respective color channel from imagenet dataset
  img[:, :, 0] -= 103.939
  img[:, :, 1] -= 116.779
  img[:, :, 2] -= 123.68

  # Convert image to tensor
  img_tensor = tf.convert_to_tensor(img)

  # Resize to 224 x 224 x 3 and add a new axis for batch size
  img_resized = tf.image.resize(img_tensor, (224,224))
  img_resized = img_resized[tf.newaxis, :]

  return img_resized, orig_shape


def tensor2img(tensor, orig_shape):
  """
  Inverse method for preprocessing function (img2tensor). This method 
  takes an image created by the network and adapts the values such that it 
  is an array in RGB colorspace, value range [0,255]. Also, the image is 
  resized to its original shape.

  tensor: (tensor) input image as tensor
  orig_shape: (array) shape of original content image
  return: (array) image for plotting
  """

  # Convert tensor to array
  img = np.array(tensor)

  # Inverse operation to zero-centering in preprocessing function 
  # (image in BGR color space)
  img[:, :, :, 0] += 103.939
  img[:, :, :, 1] += 116.779
  img[:, :, :, 2] += 123.68

  # All values must be in range [0,255], clip in case of rounding errors
  img_clipped = np.clip(img, 0., 255.)

  # Convert image to 3-channel RGB image
  img_rgb = img_clipped[0, :, :, ::-1].astype(np.uint8)

  # Resize such that size matches that of the original image
  img_resized = cv2.resize(img_rgb, (orig_shape[1], orig_shape[0]))

  return img_resized

Testing the preprocessing pipeline:

In [ ]:
# Define path to test image
img_path = os.path.join(path_content, 'puppy.jpg')

# Preprocessing of given image
img_tensor, orig_shape = img2tensor(img_path)

# Test if image actually is a tensor
assert tf.is_tensor(img_tensor), 'Image is not a tensor.'

# Test if conversion back works as well
img = tensor2img(img_tensor, orig_shape)

# Show image to check if transfer back worked.
plt.imshow(img)
plt.axis('off')
plt.show()

##**3 Model**

In the orginal the VGG-network is used. VGG is a Convolutional Neural Network (CNN) for object detection. We will first inspect the original model and then define our own model based on the pretrained layers (imagenet) of the VGG19 network.

###**3.1 Inspect VGG19**

A pretrained version of VGG19 is available from the keras applications. We load the model with the imagenet weights. We also specify that we would like to use average pooling (indicated by the parameter *pooling='avg'*) because this variant is used in the original paper. Another variant would be *pooling='max'* for max-pooling.

In [ ]:
# Load the model
model_vgg19 = tf.keras.applications.VGG19(include_top=False,
                                          weights='imagenet',
                                          pooling='avg')

# Inspect the layers.
for layer in model_vgg19.layers:
  print(layer.name)

We can observe that the model consists of an input layer, followed by five blocks and finished off by a global average pooling layer. Each block consists of four convolutional layers, finished with a pooling layer.

###**3.2 Define Model**

In [ ]:
class StyleModel(Model):

  def __init__(self, number_content_layers, layer_names):
    """
    Initializes a Style Transfer Model from layers of the VGG19 network.
    number_content_layers: (int) Number of used content layers
    layer_names: (list) list of strings with layer names (that must be 
        layer names from VGG19)
    """
    super(StyleModel, self).__init__()

    # Using the pretrained VGG19 model (on imagenet) with average pooling 
    # layers as in the original paper.
    vgg19 = tf.keras.applications.VGG19(include_top=False,
                                        weights='imagenet',
                                        pooling='avg')
    
    # Store to access the correct layers later. All other layers will be 
    # style layers
    self.number_content_layers = number_content_layers
    
    # Content and style layers respectively
    self.content_layers = [vgg19.get_layer(layer).output 
                           for layer in layer_names[:number_content_layers]]
    self.style_layers = [vgg19.get_layer(layer).output 
                         for layer in layer_names[number_content_layers:]]
    
    # Concatenate to get all output layers
    self.output_layers = self.content_layers + self.style_layers 

    # Build model
    self.model = Model([vgg19.input], self.output_layers)

  def call(self, img):
    """
    Processes a given image with the Style Transfer model.
    img: (tensor) input image, zero-centered, BGR in range [0, 255] 
    return: (list) list of outputs from each layer
    """

    # Get list of outputs
    outputs = self.model(img)

    return outputs

###**3.3 Define Gram Matrix and Loss Functions**

The style transfer is achieved through a very specific loss. The overall loss consists of the sum of weighted content and style losses. The formula for the overall loss is as follows:
<center>$L_{total}(\vec{p}, \vec{a}, \vec{x}) = \alpha * L_{content}(\vec{p}, \vec{x}) + \beta * L_{style}(\vec{a}, \vec{x})$</center>

with:
* $\vec{p} = $ content image/ photograph
* $\vec{a} = $ artwork image
* $\vec{x} = $ generated image
* $\alpha = $ weight for the content loss
* $\beta = $ weight for the style loss


$\alpha$ and $\beta$ will be provided by the user when running the style transfer. In the following sections, there will be a detailed description on how exactly the content and style losses are calculated, respectively.

####**3.3.1 Gram Matrix**

The Gram Matrix is necessary for the calculation of the style loss. The idea of the gram matrix is to get the correlation of the filter responses for each layer. Those feature correlations are provided through the inner product of the vectorized feature maps (for each layer). So, the gram matrix has a shape of $(number\_channels * number\_channels)$.

The formula for the gram matrix is as follows:
<center>$G_{ij}^l = \sum_k F_{ik}^l * F_{jk}^l$</center>

with:
* $l = $ current layer
* $G^l = $ gram matrix in layer $l$
* $F^l = $ feature map in layer $l$

In [ ]:
def gram_matrix(feature_maps, num_channels):
  """
  Defines the gram matrix for a given layer.

  feature_maps: feature maps from one layer
  num_channels: (int) number of feature maps in given layer
  return: gram-matrix of shape (num_channels x num_channels)
  """

  # Vectorize feature maps
  feature_maps_vectorized = tf.reshape(feature_maps, [-1, num_channels])

  # Multiply vectorized 'feature maps' to themselves to get the gram matrix 
  # of shape (num_channels x num_channels).
  gram_matrix = tf.matmul(feature_maps_vectorized, feature_maps_vectorized, \
                          transpose_a=True)

  return gram_matrix

####**3.3.2 Style loss**

The style loss is the mean squared error of the gram matrices of output and target for each layer Outputs and targets are the outputs from the model of the generated image (output) and the original style image (target) for all relevant style layers.

The loss is then computed by iterating over the outputs and targets for each style layer and calculating the error for those layers respectively. Each 'layer-error' is then weighted by the number of style layers.

To get the difference for original and style images' outputs in the specific layer, the gram matrices are calculated for both. The sum of squared difference is then weighted to get the mean. The full formula is as follows.

<center>$L_{style}(\vec{a}, \vec{x}) = \sum_{l=0}^L w_l * E_l$</center>

with
* $\vec{a} = $ original image/ artwork image
* $\vec{x} = $ generated image
* $w = $ weighting factor 1 / number of style layers
* and

<center>$E_l = \dfrac{1}{4 * N_l^2 * M_l^2} * \sum_{i, j} (G_{ij}^l - A_{ij}^l)^2$</center>

with
* $L =$ all layers
* $N_l =$ number of feature maps in layer $l$
* $M_l =$ size of feature maps (its height * its width) in layer $l$
* $A^l =$ gram matrix of target image in layer $l$
* $G^l =$ gram matrix of generated image in layer $l$


In [ ]:
def loss_style(outputs, targets):
  """
  Calculates the style loss. The style loss is the weighted mean squared 
  error between the output gram matrix and target gram matrix of each 
  respective layer.

  outputs: layer activations for generated image
  targets: layer activations for target style image
  return: (float) style loss
  """

  # Initialize mean-squared-error (mse) loss.
  mse = 0.

  # Check if function was called with correct parameters. 
  assert len(outputs) == len(targets), 'Outputs and targets must have the \
      same length, which refers to the same amount of layers.'
  
  # Iterate over all output and target feature maps.
  for output, target in zip(outputs, targets):

    assert np.array(output).shape == np.array(target).shape, 'Shape mismatch \
        of output and target feature maps.'

    # Get shape info: h -> height, w -> width, c -> number of channels
    _, h, w, c = np.array(output.shape).astype(np.uint32)

    # Calculate gram matrices
    gram_outputs = gram_matrix(output, c)
    gram_targets = gram_matrix(target, c)
    
    # Calculate error. Error is weighted first to get the mean error and then
    # by the number of style layers, which is represented by 'len(outputs)'.
    weighting_factor = 1 / (4. * c**2 * (h*w)**2)
    error = weighting_factor * \
        tf.reduce_sum(tf.square(gram_outputs - gram_targets))
    mse += (1. / len(outputs)) * error

  return mse

####**3.3.3 Content Loss**

The content loss is the mean squared error over the feature representations of the output and target per layer.

<center>$L_{content}(\vec{p}, \vec{x}, l) = \frac{1}{2} * \sum_{i,j} (F_{ij}^l - P_{ij}^l)^2$</center>

with
* $\vec{p} = $ content image/ photograph
* $\vec{x} = $ generated image
* $l = $ current layer $l$
* $P^l = $ feature representation in layer $l$ of content image
* $F^l = $ feature representation in layer $l$ of generated image


In [ ]:
def loss_content(outputs, targets):
  """
  Calculates the mean squared error (MSE) for the content activations.

  outputs: layer activations of generated image
  targets: layer activations of target content image
  return: Mean squared error
  """

  assert len(outputs) == len(targets), 'Length of content outputs and \
    targets must match!'

  # Weight by number of layers
  weight = 1 / len(outputs)

  # MSE is the mean over squared deviation of output and target.
  mse = 0
  for output, target in zip(outputs, targets):

    # Add up weighted errors
    mse += weight * tf.reduce_mean(tf.square(output - target))

  return mse

##**4 Running the Style Transfer**

In style transfer, there is no real 'training' phase but rather a phase where the content image iteratively gets adapted such that it takes the style of the style image. First

###**4.1 Helper Functions**

In [ ]:
def prep_images(content_image_name, style_image_name):
  """
  Function to load and preprocess exactly one content and style image.

  content_image_name: (string) file name of the content image
  style_image_name: (string) file name of the style image
  return:
    image_content: (tensor) preprocessed content image
    image_style: (tensor) preprocessed style image
    image_content_shape: (array) original shape of the content image
  """

  # Load content image and store its original shape
  image_content, image_content_shape = \
      img2tensor(os.path.join(path_content, content_image_name))

  # Load style image
  image_style, _ = img2tensor(os.path.join(path_style, style_image_name))

  return image_content, image_style, image_content_shape

In [ ]:
def clip_values(img):
  """
  Clips the values of a tensor image to the range of the respective color 
  channel. This is necessary because of the zero-centering of each image in 
  the preprocessing step.

  img: (tensor) tensor-image to clip the values
  return: (tensor) clipped version of tensor-image
  """

  norm_means = np.array([103.939, 116.779, 123.68])
  min_vals = -norm_means
  max_vals = 255 - norm_means   

  return tf.clip_by_value(img, clip_value_min=min_vals, clip_value_max=max_vals)

In [ ]:
def visualize_progress(image_names):
  """
  Visualizes progress of style transfer for the latest combination. 
  (Images in generated image folder will be overwritten in each run to prevent 
  memory issues.)

  image_names: (list) list of strings with all image names of intermediate 
    images
  return: None
  """

  # Path to generated images
  gen_path = os.path.join('/content', 'images', 'generated')

  # Iterate over all saved image names.
  for img_name in image_names:

    # Load image from directory of generated images
    img = plt.imread(os.path.join(gen_path, img_name))

    # Wait after each plot and clear old plot to get progress plot
    time.sleep(1)
    clear_output(wait=True)

    # Show image
    plt.imshow(img)
    plt.axis('off')
    plt.title('Iteration: ' + img_name.split('.')[0])
    plt.show()

###**4.2 Define Model with Specific Layers and Default Parameters**

For the content layers of our model for artistic style transfer, we chose 'block4_conv2' as in the original paper.

For the style layers, we chose the variant from the original paper. From each block, the first convolutional layer is used in the style transfer model.

In [ ]:
# Content layers: Second convolutional layer from the forth block.
layers_content = ['block4_conv2'] 

# Style layers: First convolutional layer of each of the five blocks
layers_style = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

assert len(layers_content) != 0, 'There must be at least one content layer.'
assert len(layers_style) != 0, 'There must be at least one style layer.'

# Append the layer names to define the full model.
layers_complete = layers_content + layers_style

In [ ]:
# Set model with the previously defined layers
model = StyleModel(number_content_layers=len(layers_content), 
                   layer_names=layers_complete)

# Model should not be trainable
for layer in model.layers:
    layer.trainable = False

# --- Default Parameters ---

# Optimizer
optimizer = tf.optimizers.Adam(learning_rate=4)

# Weights
weight_content_default = 1
weight_style_default = 1e-2

# Image names
image_content_name = 'neckarfront.jpg'
image_style_name = 'composition_vii.jpg'
save_name_default = image_content_name.split('.')[0] + '-' + image_style_name

# Load images
image_content_default, image_style_default, image_content_default_shape = \
    prep_images(image_content_name, image_style_name)

###**4.3 Style Transfer Function**

In [ ]:
def transfer_artistic_style(iterations=300, 
                            content=image_content_default, 
                            style=image_style_default, 
                            optimizer=optimizer, 
                            weight_content=weight_content_default, 
                            weight_style=weight_style_default,
                            content_orig_shape=(224,224,3),
                            save_name='final.jpg',
                            visualize_intermediate=False):
  """
  This function runs the style transfer on a given content image with a given 
  style image.

  iterations: (int) number of iterations of style transform
  content: (tensor) preprocessed content image
  style: (tensor) preprocessed style image
  optimizer: optimizer for model
  weight_content: (float) weight for the content loss
  weight_style: (float) weight for the style loss
  content_orig_shape: (array) shape of the original content image
  save_name: (string) name of the result image
  visualize_intermediate: (bool) whether to visualize during the transfer 
    progress
  return:
    (list) list of strings containing the names of the intermediate images
  """
  
  print('Start')

  # Create an image to apply the operations to (like content image)
  image_generated = tf.Variable(content, dtype=np.float32)
  img_gen = tensor2img(image_generated, content_orig_shape)

  # Get the content layers (first layer returned by model)
  target_content = model(content)[:model.number_content_layers]

  # Get the style layers (all layers except the first for our case)
  target_style = model(style)[model.number_content_layers:]

  # Initialize list to store image names for plotting the progress later.
  intermediate_img_names = []

  print('Got target images, start optimizing.')

  start_time = time.time()
  time_intermediate = 0

  # Iteratively adapt the generated image to match the style image.
  for i in range(iterations):

    # Compute gradients
    with tf.GradientTape() as tape:

      # Process image with model
      output = model(image_generated)

      # Again, output of the first layer is content output, all other layers 
      # are style outputs in our case
      output_content = output[:model.number_content_layers]
      output_style = output[model.number_content_layers:]

      # Calculate loss with given weights
      content_loss = loss_content(output_content, target_content)
      style_loss = loss_style(output_style, target_style)
      loss = (weight_content * content_loss) + (weight_style * style_loss)

    # Gradients with respect to generated image
    gradients = tape.gradient(loss, image_generated)

    # Generate optimized image (apply gradients to image)
    optimizer.apply_gradients([(gradients, image_generated)])

    # Assign new values image, but clip values before if necessary
    image_generated.assign(clip_values(image_generated))

    time_intermediate += time.time() - start_time - time_intermediate

    # Keep track of the progress. The progress information will be given 
    # 20 times: some information and (optionally) an intermediate image 
    # will be printed/ plotted.
    if i != 0 and int(i % (iterations/20)) == 0:
      
      # Print information
      print('Iteration: ', str(i), '--- Time passed: ', 
            round(time_intermediate, 2), 'seconds --- Loss: ', 
            np.array(loss).astype(np.float32), '--- Content Loss: ', 
            np.array(content_loss).astype(np.float32), '--- Style Loss: ', 
            np.array(style_loss).astype(np.float32))
      
      # Convert tensor image back to 'normal' image
      img_gen = tensor2img(image_generated, content_orig_shape)

      # Convert to BGR, save image with cv2, and keep track of names of 
      # intermediate images.
      intermediate_img = cv2.cvtColor(img_gen, cv2.COLOR_RGB2BGR)
      intermediate_img_names.append(str(i) + '.jpg')
      cv2.imwrite(os.path.join(path_generated, str(i) + '.jpg'), \
                  intermediate_img)

      if visualize_intermediate:
        # Show intermediate result
        plt.imshow(img_gen)
        plt.axis('off')
        plt.show()
  
  # Save result image locally
  final_img = cv2.cvtColor(img_gen, cv2.COLOR_RGB2BGR)
  cv2.imwrite(os.path.join(path_generated, save_name), final_img)

  # Print time for complete otimization process
  print('Finished', str(iterations), 'iterations in', 
        round(time.time() - start_time, 2), 'seconds.')

  return np.array(intermediate_img_names)


In [ ]:
img_names = transfer_artistic_style(2, 
                                    image_content_default, 
                                    image_style_default, 
                                    optimizer, 
                                    weight_content_default,
                                    weight_style_default,
                                    image_content_default_shape, 
                                    save_name_default,
                                    visualize_intermediate=True)

In [ ]:
# Run visualization code
visualize_progress(img_names)

##**5 Results**

To get all results, we first run the style transfer on all possible combinations on the content images we chose and then on all combinations from the original paper. The names of the result images are stored and can later be visualized.

In [ ]:
def run_all_combinations(content_images, 
                         style_images, 
                         optimizer, 
                         weight_content, 
                         weight_style, 
                         num_iterations=1000):
  """
  Runs style transfer on all possible combinations of given content and 
  style images.

  content_images: (list) list of strings of content image file names
  style_images: (list) list of strings of style image file names
  return: (list) list of strings with result image names
  """
  save_names = []

  # Iterate over content image
  for content_name in content_images:

    # For each content image, get the combination with each style image
    for style_name in style_images:

      # Keep track of progress
      print('Next image...')

      # Load and preprocess images
      image_content, image_style, image_content_shape = \
          prep_images(content_name, style_name)

      # Define name to save result image
      save_name = content_name.split('.')[0] + '-' + style_name
      save_names.append([content_name, style_name, save_name])

      # Run artistic style transfer
      transfer_artistic_style(num_iterations, 
                              image_content, 
                              image_style, 
                              optimizer, 
                              weight_content, 
                              weight_style, 
                              image_content_shape,
                              save_name)
    
  return save_names

In [ ]:
def plot_combinations(save_names):
  """
  Plots all combinations and the original images.

  save_names: (list) list of list that contains for each combination 
      the name of the content image, name of the style image and name of 
      combined images (transferred style)
  return: None
  """

  # Iterate over all result image names
  for save_name in save_names:
    
    # Load images from their directory
    img_content = plt.imread(os.path.join(path_content, save_name[0]))
    img_style = plt.imread(os.path.join(path_style, save_name[1]))
    img_combined = plt.imread(os.path.join(path_generated, save_name[2]))

    _, ax = plt.subplots(1, 3, figsize=(8,2))

    # Plot images
    ax[0].imshow(img_content), ax[0].axis('off')
    ax[1].imshow(img_style), ax[1].axis('off')
    ax[2].imshow(img_combined), ax[2].axis('off')

    plt.suptitle(TITLE_DICT[save_name[0]] + ' + ' + TITLE_DICT[save_name[1]])
    plt.show()


###**5.1 Results on Selected Content Images**

Here are the results of all possible combinations of the puppy, sunflower, and mountain content images with the style images: the scream, the starry night, Caféterrasse am Abend, and James Rizzy style.

The first cell will run the code for all possible combinations and the second cell will visualize the original content and style image and the generated image next to each other for comparison.

In [ ]:
# Define the our content image names and the the style images we would like
content_images = ['puppy.jpg', 'sunflower.jpg', 'mountain.jpg']
style_images = ['the_scream.jpg', 
                'cafeterrasse_am_abend.jpg', 
                'starry_night.jpg', 
                'rizzy.jpg']

# Run style transfer on all those possible combinations
save_names_own_content_imgs = run_all_combinations(content_images, 
                                                   style_images,
                                                   optimizer,
                                                   weight_content_default,
                                                   weight_style_default,
                                                   2)

In [ ]:
# Visualization
plot_combinations(save_names_own_content_imgs)

###**5.2 Results on Original Combinations**

In [ ]:
# Define content and style images from original paper
content_images = ['neckarfront.jpg']
style_images = ['starry_night.jpg', 
                'composition_vii.jpg', 
                'the_scream.jpg', 
                'femme_nue_assise.jpg', 
                'shipwreck.jpg']

# Run style transfer on all combinations from the original paper
save_names_original_paper = run_all_combinations(content_images,
                                                 style_images,
                                                 optimizer,
                                                 weight_content_default,
                                                 1e-2,
                                                 2)

In [ ]:
plot_combinations(save_names_original_paper)

##**6 Comparison to Original Paper**

Please refer to our paper [here](https://github.com/marisawodrich/style-transfer/blob/main/style_transfer_not_finished.pdf) for a thorough analysis of our results and analyses and a comparison of those to the orgininal paper by Gatys and colleagues.

##**References**

* Gatys, L. A., Ecker, A. S., & Bethge, M. (2015). A neural algorithm of artistic style. arXiv preprint arXiv:1508.06576.

* Neural Style Transfer with tf.keras. https://colab.research.google.com/github/tensorflow/models/blob/master/research/nst_blogpost/4_Neural_Style_Transfer_with_Eager_Execution.ipynb#scrollTo=sElaeNX-4Vnc [accessed 2021-04-03].